# 1.0 Load CRV2 GEX Data

MTX -> Sparse in-memory Format -> DataFrame (dense)

In [1]:
import pandas as pd
import helper_functions_10x as hf

from typing import Dict

# explicitly declare we're using a dictionary of DataFrames
df:Dict[str,pd.DataFrame] = {}

# working on loading

In [13]:
import pandas as pd
from scipy import io
from scipy import sparse
from ast import literal_eval as make_tuple

In [18]:
inst_path = '../data/pbmc3k_filtered_gene_bc_matrices_v2-cr/'

# matrix
Matrix = io.mmread( inst_path + 'matrix.mtx')
mat = Matrix.todense()

# genes
filename = inst_path + 'genes.tsv'
f = open(filename, 'r')
lines_genes = f.readlines()
f.close()



In [19]:
# genes
filename = inst_path + 'genes.tsv'
f = open(filename, 'r')
lines_genes = f.readlines()
f.close()

# make unique gene names
#############################
gene_list = [x.strip().split('\t') for x in lines_genes]

# find non-unique initial gene names
ini_names = [x[1] for x in gene_list]

ini_name_count = pd.Series(ini_names).value_counts()
duplicate_names = ini_name_count[ini_name_count > 1].index.tolist()
new_names = [x[1] if x[1] not in duplicate_names else x[1] + '_' + x[0] for x in gene_list]

In [29]:
print(len(new_names))
print(len(list(set(new_names))))

32738
32738


In [33]:
def load_crv2_gene_matrix(inst_path):
    '''
    Loads gene expression data from 10x in sparse matrix format and returns a
    Pandas dataframe
    '''

    import pandas as pd
    from scipy import io
    from scipy import sparse
    from ast import literal_eval as make_tuple

    # matrix
    Matrix = io.mmread( inst_path + 'matrix.mtx')
    mat = Matrix.todense()

#     # genes
#     filename = inst_path + 'genes.tsv'
#     f = open(filename, 'r')
#     lines = f.readlines()
#     f.close()
    
    # genes
    filename = inst_path + 'genes.tsv'
    f = open(filename, 'r')
    lines_genes = f.readlines()
    f.close()

    # make unique gene names
    #############################
    gene_list = [x.strip().split('\t') for x in lines_genes]

    # find non-unique initial gene names
    ini_names = [x[1] for x in gene_list]

    ini_name_count = pd.Series(ini_names).value_counts()
    duplicate_names = ini_name_count[ini_name_count > 1].index.tolist()
    genes = [x[1] if x[1] not in duplicate_names else x[1] + '_' + x[0] for x in gene_list]    

    
#     # add unique id only to duplicate genes
#     ###########################################################    
#     ini_genes = []
#     for inst_line in lines:
#         inst_line = inst_line.strip().split()
#         if len(inst_line) > 1:
#           inst_gene = inst_line[1]
#         else:
#           inst_gene = inst_line[0]
#         ini_genes.append(inst_gene)

#     gene_name_count = pd.Series(ini_genes).value_counts()
#     duplicate_genes = gene_name_count[gene_name_count > 1].index.tolist()

#     print('dup genes')
#     print(duplicate_genes)

#     dup_index = {}
#     genes = []
#     for inst_row in ini_genes:

#       # add index to non-unique genes
#       if inst_row in duplicate_genes:

#         # calc_non-unque index
#         if inst_row not in dup_index:
#           dup_index[inst_row] = 1
#         else:
#           dup_index[inst_row] = dup_index[inst_row] + 1

#         new_row = inst_row + '_' + str(dup_index[inst_row])

#       else:
#         new_row = inst_row

#       genes.append(new_row)
    
#     ###########################################################

    # barcodes
    filename = inst_path + 'barcodes.tsv'
    f = open(filename, 'r')
    lines = f.readlines()
    f.close()

    cell_barcodes = []
    for inst_bc in lines:
        inst_bc = inst_bc.strip().split('\t')

        # remove dash from barcodes if necessary
        if '-' in inst_bc[0]:
          inst_bc[0] = inst_bc[0].split('-')[0]

        cell_barcodes.append(inst_bc[0])

    # parse tuples if necessary
    try:
        cell_barcodes = [make_tuple(x) for x in cell_barcodes]
    except:
        pass

    try:
        genes = [make_tuple(x) for x in genes]
    except:
        pass

    # make dataframe
    df = pd.DataFrame(mat, index=genes, columns=cell_barcodes)
    
    ### change ouput to feature_data

    return df

### Load GEX Data into DataFrame

In [36]:
df = {}

In [37]:
df['gex-ini'] = load_crv2_gene_matrix('../data/pbmc3k_filtered_gene_bc_matrices_v2-cr/')

In [38]:
rows = df['gex-ini'].index.tolist()

In [39]:
found_rows = [x for x in rows if '7SK' in x]
print(found_rows)

['7SK_ENSG00000249352', '7SK_ENSG00000254144', '7SK_ENSG00000260682', '7SK_ENSG00000232512']


In [40]:
rows = df['gex-ini'].index.tolist()
print(len(rows))
print(len(list(set(rows))))

32738
32738


In [ ]:
df['gex-ini'].shape

In [ ]:
df['gex-ini'].head()

### Checking Ensemble Gene IDS

In [ ]:
filename = '../data/pbmc3k_filtered_gene_bc_matrices_v2-cr/genes.tsv'
f = open(filename, 'r')
lines = f.readlines()
lines[0]

In [ ]:
ens_list = []
gene_list = []
for inst_line in lines:
    inst_line = inst_line.strip().split('\t')
    ens_list.append(inst_line[0])
    gene_list.append(inst_line[1])

In [ ]:
print(len(ens_list))
print(len(list(set(ens_list))))

In [ ]:
print(len(gene_list))
print(len(list(set(gene_list))))